# **The Code (3 state Kalman)**

In [ ]:
import numpy as np
import time
import cv2

from src.computerVision import correctPerspectiveStream, findCorners, getPerspectiveMatrix, findThymio, findGlobalObstacles, findGoal
from src.kalmanOld import estimatePosition, inverseSpeedConversion, r11, r22, r33, Cl, Cr, R, L
from src.pathPlanning import buildGraph
from src.robotControl import robotController, checkForObstacles

XYMIRROR = False
IMAGE_WIDTH = 1920
IMAGE_HEIGHT = 1080
POSITION_THRESHOLD = 50
KIDNAPPING_THRESHOLD = 100
KIDNAPPING_TIME = 5


In [ ]:
# Note: This function was provided by the course Basics Of Mobile Robotics by Prof. Francesco Mondada
# Source: https://moodle.epfl.ch/course/view.php?id=15293

import tdmclient.notebook
await tdmclient.notebook.start()



@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500, verbose=False):

    global motor_left_target, motor_right_target

    l_speed = int(l_speed)
    r_speed = int(r_speed)
    
    motor_left_target = l_speed
    motor_right_target = r_speed

In [ ]:
# Note: This function was provided by the course Basics Of Mobile Robotics by Prof. Francesco Mondada
# Source: https://moodle.epfl.ch/course/view.php?id=15293

@tdmclient.notebook.sync_var 
def horiz_sensor():
    global prox_horizontal
    return prox_horizontal

In [ ]:
# Note: This function was provided by the course Basics Of Mobile Robotics by Prof. Francesco Mondada
# Source: https://moodle.epfl.ch/course/view.php?id=15293

@tdmclient.notebook.sync_var 
def getVerticalDistance():
    global prox_ground_reflected
    return prox_ground_reflected


In [ ]:
%%run_python

# Note: This function was provided by the course Basics Of Mobile Robotics by Prof. Francesco Mondada
# Source: https://moodle.epfl.ch/course/view.php?id=15293

nf_leds_prox_h(0,0,0,0,0,0,0,0) 
nf_leds_rc(0)
nf_leds_temperature(0, 0)
nf_leds_bottom_right(0,0,0)
nf_leds_bottom_left(0,0,0)
nf_leds_top(0,0,0)
nf_leds_prox_v(0,0)


In [ ]:
#Note: these functions are defined here and not in their respective files because they call the functions that interact with the robot defined above

def checkForKidnap():
    ver=getVerticalDistance()
    if ver[0] < KIDNAPPING_THRESHOLD or ver[1] < KIDNAPPING_THRESHOLD:
        print("Kidnapped!")
        motors(0,0)
        return True
    else:
        return False

def avoidObstacle(leftOrRight, initTime, currentTime):
    motors(0,0)
    
    tTurn = 1
    tStraight = 2
    vStraight = 200

    if leftOrRight == 'right':
        l=-200
        r=200
    elif leftOrRight == 'left':
        l=200
        r=-200

    motors(l_speed=l, r_speed=r)
    if currentTime - initTime < tTurn:
        return True

    motors(l_speed=vStraight, r_speed=vStraight)
    if currentTime - initTime < tTurn + tStraight:
        return True
    

    motors(l_speed=-l, r_speed=-r)
    if currentTime - initTime < 2*tTurn + tStraight:
        return True

    motors(l_speed=vStraight, r_speed=vStraight)
    if currentTime - initTime < 2*tTurn + 2.5*tStraight:
        return True

    motors(l_speed=-l, r_speed=-r)
    if currentTime - initTime < 3*tTurn + 2.5*tStraight:
        return True

    motors(l_speed=vStraight, r_speed=vStraight)
    if currentTime - initTime < 3*tTurn + 3.5*tStraight:
        return True

    motors(l_speed=l, r_speed=r)
    if currentTime - initTime < 4*tTurn + 3.5*tStraight:
        return True

    motors(l_speed=0, r_speed=0)

    return False


In [ ]:
cap = cv2.VideoCapture(1)
previousTime = time.time()

#state control variables
correctedCam = False
environmentSetup = False
redoPath = False
avoidingObstacle = False
wasKidnapped = False
aligned = False

#initializing variables
path = []
goal = np.array([0,0])
lSpeed = 0
rSpeed = 0
P_k = np.array([[r11,0,0],[0,r22,0],[0,0,r33]])
postionHistory = []


while True:
    # Read a frame from the video stream
    key = cv2.waitKey(1)
    ret, frame = cap.read()


    if not ret:
        print("Error: failed to capture image")
        break

    if not correctedCam:
        # Map the image to the corner
        position, angle, _=findThymio(frame)
        estimateState = np.array([position[0],IMAGE_HEIGHT-position[1],angle])
        cv2.imshow('Thymio Camera', frame)
        try:
            centroids = findCorners(frame)
            perspectiveMatrix = getPerspectiveMatrix(centroids)
            correctedCam = True
        except:
            print("Fail")
            continue
            
    
    
    # Correct the perspective of the image
    if correctedCam:
        frame = correctPerspectiveStream(frame,perspectiveMatrix)
        frameToPlot = frame



    current_time = time.time()
    dt = current_time - previousTime
    if dt < 0.15:
        time.sleep(0.15-dt)

    previousTime = current_time


    previousControlVector = inverseSpeedConversion(rSpeed,lSpeed,R,L,Cr,Cl)

    position, angle, estimateState, P_k = estimatePosition(frame,previousControlVector,dt,P_k,estimateState)

    postionHistory.append(position)



    #plot the points in position history in black
    for i in range(len(postionHistory)-1):
        cv2.circle(frameToPlot, (int(postionHistory[i][0]), int(IMAGE_HEIGHT-postionHistory[i][1])), 5, (0, 0, 0), -1)

    #plot the goal
    cv2.circle(frameToPlot, (int(goal[0]), int(goal[1])), 5, (0, 0, 255), -1)

    #plot the path lines
    for i in range(len(path)-1):
        cv2.line(frameToPlot,(int(path[i][0]),int(IMAGE_HEIGHT-path[i][1])),(int(path[i+1][0]),int(IMAGE_HEIGHT-path[i+1][1])),(255,255,255),2)
        cv2.circle(frameToPlot, (int(path[i][0]), int(IMAGE_HEIGHT-path[i][1])), 5, (255, 255, 255), -1)
        
    cv2.imshow('Thymio Camera', frameToPlot)
    
    
    if checkForKidnap():
        #cambiare le condizioni
        print("kidnapped")
        lSpeed = 0
        rSpeed = 0
        wasKidnapped = True
        continue


    if wasKidnapped:
        time.sleep(KIDNAPPING_TIME)
        wasKidnapped = False
        redoPath = True
        continue


    if not avoidingObstacle:
        leftOrRight = checkForObstacles(horiz_sensor())


    if leftOrRight != None or avoidingObstacle:
        if not avoidingObstacle:
            initTime = time.time()
        avoidingObstacle = avoidObstacle(leftOrRight, initTime, time.time())
        redoPath = True
        continue
    

    if (key == ord(' ') and environmentSetup == False) or redoPath:
        # Capture an image
        imagePath = 'capturedImage.jpg'
        cv2.imwrite(imagePath, frame)

        capturedImage = cv2.imread(imagePath)
        position, angle, estimateState, P_k = estimatePosition(capturedImage,previousControlVector,dt,P_k,estimateState)

        greenObjects = findGlobalObstacles(capturedImage)

        goal = findGoal(capturedImage)
        
        print(goal)

        cv2.imshow('Thymio Camera', capturedImage)
        

        path=buildGraph(greenObjects,position,goal)
        pointCount = 0
        print(path)

        if redoPath:
            redoPath = False
            continue
        environmentSetup = True
        
        #cv2.waitKey(0)  # Wait until any key is pressed to close the image window



    if environmentSetup:

        if np.linalg.norm(position - path[pointCount]) < POSITION_THRESHOLD:
            aligned = False
            pointCount += 1
            motors(0,0)

            if pointCount == len(path):
                motors(0,0)
                print("Goal reached")
                break

            continue
        
        if not aligned:
            lSpeed,rSpeed,distanceError,angleError = robotController(path[pointCount-1],path[pointCount],position,angle,dt,alignMode=True)
            motors(lSpeed,rSpeed)
            if np.abs(angleError) < 0.1:
                aligned = True
                motors(0,0)
            continue


        lSpeed,rSpeed,distanceError,angleError = robotController(path[pointCount-1],path[pointCount],position,angle,dt,alignMode=False)
        motors(lSpeed,rSpeed)

        # Break the loop if 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
motors(0,0)

# **Sources**

Course "Basics of Mobile Robotics" by Prof. Francesco Mondada (MICRO-452): https://moodle.epfl.ch/course/view.php?id=15293

Automatic Addison: https://automaticaddison.com

mouhknowsbest (Youtube Channel): https://www.youtube.com/watch?v=aE7RQNhwnPQ

Chat-GPT4: https://chat.openai.com/

Copilot: https://copilot.github.com/

Numpy: https://numpy.org/doc/

OpenCV: https://docs.opencv.org/

Pyvisgraph: https://github.com/TaipanRex/pyvisgraph/tree/master

matplotlib: https://matplotlib.org/stable/contents.html

TDM Client: https://github.com/epfl-mobots/tdm-python/tree/main